### Data Cleaning and Preprocessing Code

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
pd.set_option('display.max_columns',None)

In [2]:
data_path="D://ML/Kaggle/hk/"

In [3]:
#races.csv contains the condition of every race
df_races=pd.read_csv(data_path+"races.csv")
#runs.csv contains horses data for every race
df_runs=pd.read_csv(data_path+"runs.csv")

In [4]:
df_races.shape, df_runs.shape

((6349, 37), (79447, 37))

In [5]:
df_races.head(2)

,race_id,date,venue,race_no,config,surface,distance,going,horse_ratings,prize,race_class,sec_time1,sec_time2,sec_time3,sec_time4,sec_time5,sec_time6,sec_time7,time1,time2,time3,time4,time5,time6,time7,place_combination1,place_combination2,place_combination3,place_combination4,place_dividend1,place_dividend2,place_dividend3,place_dividend4,win_combination1,win_dividend1,win_combination2,win_dividend2
0,0,1997-06-02,ST,1,A,0,1400,GOOD TO FIRM,40-15,485000.0,5,13.53,21.59,23.94,23.58,NaN,NaN,NaN,13.53,35.12,59.06,82.64,NaN,NaN,NaN,8,11,6.0,NaN,36.5,25.5,18.0,NaN,8,121.0,NaN,NaN
1,1,1997-06-02,ST,2,A,0,1200,GOOD TO FIRM,40-15,485000.0,5,24.05,22.64,23.70,NaN,NaN,NaN,NaN,24.05,46.69,70.39,NaN,NaN,NaN,NaN,5,13,4.0,NaN,12.5,47.0,33.5,NaN,5,23.5,NaN,NaN


In [6]:
df_runs.head(3)

,race_id,horse_no,horse_id,result,won,lengths_behind,horse_age,horse_country,horse_type,horse_rating,horse_gear,declared_weight,actual_weight,draw,position_sec1,position_sec2,position_sec3,position_sec4,position_sec5,position_sec6,behind_sec1,behind_sec2,behind_sec3,behind_sec4,behind_sec5,behind_sec6,time1,time2,time3,time4,time5,time6,finish_time,win_odds,place_odds,trainer_id,jockey_id
0,0,1,3917,10,0.0,8.00,3,AUS,Gelding,60,--,1020.0,133,7,6,4,6,10.0,NaN,NaN,2.0,2.0,1.50,8.00,NaN,NaN,13.85,21.59,23.86,24.62,NaN,NaN,83.92,9.7,3.7,118,2
1,0,2,2157,8,0.0,5.75,3,NZ,Gelding,60,--,980.0,133,12,12,13,13,8.0,NaN,NaN,6.5,9.0,5.00,5.75,NaN,NaN,14.57,21.99,23.30,23.70,NaN,NaN,83.56,16.0,4.9,164,57
2,0,3,858,7,0.0,4.75,3,NZ,Gelding,60,--,1082.0,132,8,3,2,2,7.0,NaN,NaN,1.0,1.0,0.75,4.75,NaN,NaN,13.69,21.59,23.90,24.22,NaN,NaN,83.40,3.5,1.5,137,18


In [7]:
#Selecting columns which are present before the race starts so the model can be trained on that information
df_races=df_races[['race_id','date','venue','race_no','config','surface','distance','going','horse_ratings','prize','race_class']]
df_runs=df_runs[['race_id','horse_id','horse_age','horse_country','horse_type','horse_rating','declared_weight','actual_weight','draw','win_odds','place_odds','trainer_id','jockey_id','result']]

In [8]:
df_races.head(2)

,race_id,date,venue,race_no,config,surface,distance,going,horse_ratings,prize,race_class
0,0,1997-06-02,ST,1,A,0,1400,GOOD TO FIRM,40-15,485000.0,5
1,1,1997-06-02,ST,2,A,0,1200,GOOD TO FIRM,40-15,485000.0,5


In [9]:
df_runs.head(3)

,race_id,horse_id,horse_age,horse_country,horse_type,horse_rating,declared_weight,actual_weight,draw,win_odds,place_odds,trainer_id,jockey_id,result
0,0,3917,3,AUS,Gelding,60,1020.0,133,7,9.7,3.7,118,2,10
1,0,2157,3,NZ,Gelding,60,980.0,133,12,16.0,4.9,164,57,8
2,0,858,3,NZ,Gelding,60,1082.0,132,8,3.5,1.5,137,18,7


### Modeling data and OOT data
Column "race_id" is unique indentifer for both data. We will divide data into two parts based on date: Modeling data and OOT data. OOT data will never be seen by model and is the latest data in whole dataset

Modeling data: Races from min date (1997-06-02) to 2005-07-31 (will be used in modeling)

OOT data: Races after 2005-08-01 (will make betting strategy on this)

In [10]:
modeling_races = df_races[df_races['date']<"2005-08-01"].reset_index(drop=True)
oot_races = df_races[df_races['date']>="2005-08-01"].reset_index(drop=True)

In [11]:
print(f"Max date in modeling race data: {modeling_races.date.max()} \nMin date in modeling race data: {modeling_races.date.min()} \nMax date in OOT race data: {oot_races.date.max()} \nMin date in OOT race data: {oot_races.date.min()}")

Max date in modeling race data: 2005-07-30 
Min date in modeling race data: 1997-06-02 
Max date in OOT race data: 2005-08-28 
Min date in OOT race data: 2005-08-03


In [12]:
#Label encoding categorical variables
venue_encoder = LabelEncoder()
config_encoder = LabelEncoder()
going_encoder = LabelEncoder()
horse_ratings_encoder = LabelEncoder()

modeling_races['venue']=venue_encoder.fit_transform(modeling_races['venue'])
modeling_races['config']=config_encoder.fit_transform(modeling_races['config'])
modeling_races['going']=going_encoder.fit_transform(modeling_races['going'])
modeling_races['horse_ratings']=horse_ratings_encoder.fit_transform(modeling_races['horse_ratings'])

oot_races['venue']=venue_encoder.transform(oot_races['venue'])
oot_races['config']=config_encoder.transform(oot_races['config'])
oot_races['going']=going_encoder.transform(oot_races['going'])
oot_races['horse_ratings']=horse_ratings_encoder.transform(oot_races['horse_ratings'])

In [13]:
#converting date column to number of days from referrence date 1995-01-01
modeling_races['date']=(pd.to_datetime(modeling_races['date'])- pd.to_datetime('1995-01-01'))/ np.timedelta64(1, 'D')
oot_races['date']=(pd.to_datetime(oot_races['date'])- pd.to_datetime('1995-01-01'))/ np.timedelta64(1, 'D')

In [14]:
df_runs.dtypes

race_id              int64
horse_id             int64
horse_age            int64
horse_country       object
horse_type          object
horse_rating         int64
declared_weight    float64
actual_weight        int64
draw                 int64
win_odds           float64
place_odds         float64
trainer_id           int64
jockey_id            int64
result               int64
dtype: object

In [15]:
#Dropping noise data
df_runs.dropna(subset=['horse_country','horse_type'],how='any',inplace=True)
df_runs=df_runs[df_runs['draw']<15]# Max possible draw is 14

In [16]:
modeling_runs = df_runs[df_runs['race_id'].isin(list(set(modeling_races['race_id'].tolist())))].reset_index(drop=True)
oot_runs = df_runs[df_runs['race_id'].isin(list(set(oot_races['race_id'].tolist())))].reset_index(drop=True)

In [17]:
#Label encoding categorical variables in horse data
horse_country_encoder = LabelEncoder()
horse_type_encoder = LabelEncoder()

modeling_runs['horse_country']=horse_country_encoder.fit_transform(modeling_runs['horse_country'])
modeling_runs['horse_type']=horse_type_encoder.fit_transform(modeling_runs['horse_type'])

oot_runs['horse_country']=horse_country_encoder.transform(oot_runs['horse_country'])
oot_runs['horse_type']=horse_type_encoder.transform(oot_runs['horse_type'])

In [18]:
#custom function used in rearranging columns, so that results column comes in the end
def group_horse_and_result(element):
    if element[0] == 'result':
        return 100 + element[1]
    else:
        return element[1]   

In [19]:
modeling_runs = modeling_runs.pivot(index='race_id', columns='draw', values=modeling_runs.columns[2:])
oot_runs = oot_runs.pivot(index='race_id', columns='draw', values=oot_runs.columns[2:])

In [20]:
rearranged_columns = sorted(list(modeling_runs.columns.values), key=group_horse_and_result)
modeling_runs = modeling_runs[rearranged_columns]
oot_runs = oot_runs[rearranged_columns]

In [22]:
#Now we have horses data on race level for modeling and oot
modeling_runs.head(2)

horse_age horse_country horse_type horse_rating declared_weight  \
draw           1             1          1            1               1    
race_id                                                                   
0             3.0          14.0        3.0         60.0          1089.0   
1             3.0           1.0        3.0         60.0          1059.0   

        actual_weight draw win_odds place_odds trainer_id jockey_id horse_age  \
draw               1    1        1          1          1         1         2    
race_id                                                                         
0               120.0  1.0      5.4        1.7      128.0     138.0       3.0   
1               121.0  1.0     10.0        2.6      109.0     145.0       3.0   

        horse_country horse_type horse_rating declared_weight actual_weight  \
draw               2          2            2               2             2    
race_id                                                                       
0                 1.0        3.0         60.0          1170.0         128.0   
1                11.0        3.0         60.0          1057.0         121.0   

        draw win_odds place_odds trainer_id jockey_id horse_age horse_country  \
draw      2        2          2          2         2         3             3    
race_id                                                                         
0        2.0     12.0        3.6       47.0     183.0       3.0          11.0   
1        2.0     28.0        6.3       54.0      34.0       3.0           1.0   

        horse_type horse_rating declared_weight actual_weight draw win_odds  \
draw            3            3               3             3    3        3    
race_id                                                                       
0              3.0         60.0          1135.0         123.0  3.0      8.6   
1              3.0         60.0          1095.0         103.0  3.0     21.0   

        place_odds trainer_id jockey_id horse_age horse_country horse_type  \
draw            3          3         3         4             4          4    
race_id                                                                      
0              2.5        4.0     155.0       3.0           1.0        3.0   
1              7.4       38.0      40.0       3.0           1.0        0.0   

        horse_rating declared_weight actual_weight draw win_odds place_odds  \
draw              4               4             4    4        4          4    
race_id                                                                       
0               60.0          1027.0         113.0  4.0     11.0        3.9   
1               60.0          1110.0         108.0  4.0     47.0       16.0   

        trainer_id jockey_id horse_age horse_country horse_type horse_rating  \
draw            4         4         5             5          5            5    
race_id                                                                        
0             38.0     162.0       3.0          11.0        3.0         60.0   
1            137.0     128.0       3.0           1.0        5.0         60.0   

        declared_weight actual_weight draw win_odds place_odds trainer_id  \
draw                 5             5    5        5          5          5    
race_id                                                                     
0                1114.0         127.0  5.0      7.0        1.8       54.0   
1                1037.0         130.0  5.0      7.0        1.7       75.0   

        jockey_id horse_age horse_country horse_type horse_rating  \
draw           5         6             6          6            6    
race_id                                                             
0            34.0       3.0          11.0        3.0         60.0   
1           183.0       3.0           1.0        3.0         60.0   

        declared_weight actual_weight draw win_odds place_odds trainer_id  \
draw                 6         

In [23]:
modeling_runs.fillna(-999, inplace=True)
modeling_races.fillna(-999, inplace=True)

oot_runs.fillna(-999, inplace=True)
oot_races.fillna(-999, inplace=True)

In [24]:
#Merging horse data with race data on columns race_id
modeling_data = modeling_races.join(modeling_runs, on='race_id', how='inner')
oot_data = oot_races.join(oot_runs, on='race_id', how='inner')

C:\Users\Prashant\AppData\Local\Temp\ipykernel_9036\679092660.py:2: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  modeling_data = modeling_races.join(modeling_runs, on='race_id', how='inner')
C:\Users\Prashant\AppData\Local\Temp\ipykernel_9036\679092660.py:3: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  oot_data = oot_races.join(oot_runs, on='race_id', how='inner')


In [25]:
modeling_runs.shape, modeling_races.shape,modeling_data.shape

((6274, 168), (6275, 11), (6274, 179))

In [26]:
oot_runs.shape, oot_races.shape,oot_data.shape

((74, 168), (74, 11), (74, 179))

In [27]:
modeling_data.to_csv(data_path+"modeling_data.csv",index=False)

In [28]:
oot_data.to_csv(data_path+"oot_data.csv",index=False)